# Hybrid Execution Automatic Switching Demo

In [1]:
import snowflake.snowpark.modin.plugin
import modin.pandas as pd
import numpy as np
import datetime
import pandas as native_pd
from snowflake.snowpark.session import Session; session = Session.builder.create()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://snowbiz.okta.com/app/snowflake/exk8wfsfryJIn4IWZ2p7/sso/saml?SAMLRequest=jZJBb%2BIwEIX%2FSuQ9EzsJW1oLqFjYtqnagprQStxMYqiFYwePQ8j%2B%2BnVCWXUPrXqzxu%2FZ38yb4fWxkN6BGxBajVDgE%2BRxlelcqO0ILdOb3iXywDKVM6kVH6GGA7oeD4EVsqSTyr6pZ76vOFjPPaSAdhcjVBlFNQMBVLGCA7UZTSaPDzT0CS2NtjrTEn2wfO1gANxYR3i25CAc3pu1JcW4rmu%2FjnxttjgkhGByhZ2qlfw464%2Bup0%2F0ASb9Vu8UTr54Z%2Fsl1GkEX2GtTyKgd2m66C3mSYq8yRl1qhVUBTcJNweR8eXzwwkAHEHyNH%2B9my%2BT3z4oXW8k2%2FFMF2Vl3Wu%2BO%2BENz7HUW%2BEajmcjVO5Evocs6pe3ZrdOpnzNdHTbxNO0CjI4mAu53A%2BqK7Nq9vcxbDPkvZwTDdtEY4CKx6rN0boSCX%2F2SL8X9NMwokFASeSTi8EKeTOXo1DMds4zbIu4Fn98vbOsg2Nlif9xY37cXdYb2JjmPlb9%2BHUVlgMMoHEbKzptCu0AzPi7%2FQ%2FxR9f7sj25%2BcezhZYia7wbbQpmP48n8IOuIvLeppNSXjAhJ3luOICLSUpdTw1n1u20NRVHeHz69f%2BtHv8F&RelayState=ver%

## Example 1: Working with small/inline-created dataframe is faster

In [2]:
us_holidays = [
    ("New Year's Day", "2025-01-01"),
    ("Martin Luther King Jr. Day", "2025-01-20"),
    ("Presidents' Day", "2025-02-17"),
    ("Memorial Day", "2025-05-26"),
    ("Juneteenth National Independence Day", "2025-06-19"),
    ("Independence Day", "2025-07-04"),
    ("Labor Day", "2025-09-01"),
    ("Columbus Day", "2025-10-13"),
    ("Veterans Day", "2025-11-11"),
    ("Thanksgiving Day", "2025-11-27"),
    ("Christmas Day", "2025-12-25")
]

# Create DataFrame
df_us_holidays = pd.DataFrame(us_holidays, columns=["Holiday", "Date"])

# Convert Date column to datetime
df_us_holidays["Date"] = pd.to_datetime(df_us_holidays["Date"])

In [3]:
assert df_us_holidays.get_backend() == 'Pandas'  # with auto, we should expect this to be local

In [4]:
# Add new columns for transformations
df_us_holidays["Day_of_Week"] = df_us_holidays["Date"].dt.day_name()
df_us_holidays["Month"] = df_us_holidays["Date"].dt.month_name()

In [5]:
df_us_holidays

,Holiday,Date,Day_of_Week,Month
0,New Year's Day,2025-01-01,Wednesday,January
1,Martin Luther King Jr. Day,2025-01-20,Monday,January
2,Presidents' Day,2025-02-17,Monday,February
3,Memorial Day,2025-05-26,Monday,May
4,Juneteenth National Independence Day,2025-06-19,Thursday,June
5,Independence Day,2025-07-04,Friday,July
6,Labor Day,2025-09-01,Monday,September
7,Columbus Day,2025-10-13,Monday,October
8,Veterans Day,2025-11-11,Tuesday,November
9,Thanksgiving Day,2025-11-27,Thursday,November


In [6]:
%%time
#Note that without auto-switching, this took 2.5 min
for index, row in df_us_holidays.iterrows():
    print(f"{row['Holiday']} falls on {row['Day_of_Week']}, {row['Month']} {row['Date'].day}, {row['Date'].year}.")

New Year's Day falls on Wednesday, January 1, 2025.
Martin Luther King Jr. Day falls on Monday, January 20, 2025.
Presidents' Day falls on Monday, February 17, 2025.
Memorial Day falls on Monday, May 26, 2025.
Juneteenth National Independence Day falls on Thursday, June 19, 2025.
Independence Day falls on Friday, July 4, 2025.
Labor Day falls on Monday, September 1, 2025.
Columbus Day falls on Monday, October 13, 2025.
Veterans Day falls on Tuesday, November 11, 2025.
Thanksgiving Day falls on Thursday, November 27, 2025.
Christmas Day falls on Thursday, December 25, 2025.
CPU times: user 124 ms, sys: 6.7 ms, total: 131 ms
Wall time: 128 ms


Automatic engine switch happens when merged with large dataset.

In [7]:
df_transactions = pd.read_snowflake("REVENUE_TRANSACTIONS")

Switcheroo operation: read_snowflake cost to move to pandas: 1000 cost to stay: 250


In [8]:
df_transactions["DATE"] = pd.to_datetime(df_transactions["DATE"])

In [9]:
len(df_us_holidays), len(df_transactions)

(11, 10000000)

In [10]:
combined = pd.merge(df_us_holidays, df_transactions, left_on="Date", right_on="DATE")

BackendCostCalculator Results: Pandas:1000/1000,*Snowflake:500/10000000


Transferring data from Pandas to Snowflake ...:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
combined.get_backend()

'Snowflake'

### 💡 Automatic switching speeds up loops/iterations on small data + inline creation of dataframes

## Example 2: When data is filtered the choice of engine changes

Run the following SQL to generate a synthetic dataset with 10M rows of transactions (from 2024-2025 current date)
```sql
CREATE OR REPLACE TABLE revenue_transactions (
    Transaction_ID STRING,
    Date DATE,
    Revenue FLOAT
);

SET num_days = (SELECT DATEDIFF(DAY, '2024-01-01', CURRENT_DATE));
INSERT INTO revenue_transactions (Transaction_ID, Date, Revenue)
SELECT
    UUID_STRING() AS Transaction_ID,
    DATEADD(DAY, UNIFORM(0, $num_days, RANDOM()), '2024-01-01') AS Date,
    UNIFORM(10, 1000, RANDOM()) AS Revenue
FROM TABLE(GENERATOR(ROWCOUNT => 10000000));
```

In [12]:
# Run the following to generate a synthetic dataset with 10M rows of transactions (from 2024-2025 current date)
session.sql('''
CREATE OR REPLACE TABLE revenue_transactions (
    Transaction_ID STRING,
    Date DATE,
    Revenue FLOAT
);''').collect()
session.sql('''SET num_days = (SELECT DATEDIFF(DAY, '2024-01-01', CURRENT_DATE));''').collect()
session.sql('''INSERT INTO revenue_transactions (Transaction_ID, Date, Revenue)
SELECT
    UUID_STRING() AS Transaction_ID,
    DATEADD(DAY, UNIFORM(0, $num_days, RANDOM()), '2024-01-01') AS Date,
    UNIFORM(10, 1000, RANDOM()) AS Revenue
FROM TABLE(GENERATOR(ROWCOUNT => 10000000));
''').collect()

[Row(number of rows inserted=10000000)]

In [13]:
df_transactions = pd.read_snowflake("REVENUE_TRANSACTIONS")

Switcheroo operation: read_snowflake cost to move to pandas: 1000 cost to stay: 250


In [14]:
len(df_transactions)

10000000

Perform some operations on 10M rows with Snowflake

In [15]:
df_transactions["DATE"] = pd.to_datetime(df_transactions["DATE"])

In [16]:
df_transactions.groupby("DATE").sum()["REVENUE"]

Switcheroo operation: sum cost to move to pandas: 1000 cost to stay: 250


DATE
2024-01-01    10683751.0
2024-01-02    10763671.0
2024-01-03    10865040.0
2024-01-04    10762951.0
2024-01-05    10644397.0
                 ...    
2025-04-10    10759502.0
2025-04-11    10663488.0
2025-04-12    10678733.0
2025-04-13    10841435.0
2025-04-14    10691687.0
Freq: None, Name: REVENUE, Length: 470, dtype: float64

In [17]:
assert df_transactions.get_backend() == "Snowflake"

So far everything has been happening in Snowflake, since we are working with the full dataset (10M rows). 
Next, we demonstrate what happens when we filter the data down to a smaller dataset below our 500k threshold for automatic switching. 
We showcase two was of doing the filtering: 
- Method 1: Filtering with pandas (lazy evaluation)
- Method 2: Prefilter with SQL during dataframe creation 

In [18]:
df_transactions_filter1 = df_transactions[(df_transactions["DATE"] >= pd.Timestamp.today().date() - pd.Timedelta('7 days')) & (df_transactions["DATE"] < pd.Timestamp.today().date())]
df_transactions_filter1

,TRANSACTION_ID,DATE,REVENUE
30,8fb38415-82af-45f7-baac-8bba00b3fda4,2025-04-07,545.0
37,311bc73d-2be3-48cd-a13a-faeb8112cced,2025-04-08,980.0
75,ebbb418c-3a59-476e-ba96-ec9711c53543,2025-04-12,752.0
111,9b8622fb-7916-4923-b22b-ecc9ee81cc9c,2025-04-09,367.0
188,46ffd5c9-e5e2-4687-a159-61c645a66cf6,2025-04-12,200.0
...,...,...,...
9999675,b5001e3c-df38-4245-a7fa-f93e52733708,2025-04-12,50.0
9999763,4bdbc8bc-8d65-499b-8b31-6db25ab81657,2025-04-12,145.0
9999765,a9e1fdab-e863-4398-b86a-ac0107e4dbae,2025-04-07,548.0
9999913,7a2a3d26-8443-4b9c-80dc-9f1163bef765,2025-04-07,773.0


In this case, since the data is already in Snowflake, it stays in Snowflake even after the filtering.

In [19]:
assert df_transactions_filter1.get_backend() == "Snowflake"

In [20]:
print(f"Date range: {df_transactions_filter1['DATE'].min().date()} to {df_transactions_filter1['DATE'].max().date()}. Resulting dataset size: {len(df_transactions_filter1)}")

Date range: 2025-04-07 to 2025-04-13. Resulting dataset size: 148673


Now let's perform filtering via SQL directly, so the dataframe upon creation is small.

In [21]:
df_transactions_filter2 = pd.read_snowflake("SELECT * FROM revenue_transactions WHERE Date >= DATEADD( 'days', -7, current_date ) and Date < current_date")

Switcheroo operation: read_snowflake cost to move to pandas: 37 cost to stay: 750


Transferring data from Snowflake to Pandas ...:   0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
df_transactions_filter2

,TRANSACTION_ID,DATE,REVENUE
0,cf019985-e0c1-4e7f-992d-376eaaab5890,2025-04-08,309.0
1,e21e2c6a-4944-46fe-8d41-3072dfda6cb7,2025-04-09,650.0
2,3ca47942-5521-4c18-be17-6990a69c0f4e,2025-04-07,854.0
3,c974cf85-0bac-4098-8fa1-b74e7210a6f3,2025-04-09,485.0
4,ec1c7ccf-aaf5-4754-8482-eaf18574e147,2025-04-11,181.0
...,...,...,...
148668,14c2e5f8-76e9-4f6a-8298-07b95c4f1a76,2025-04-10,790.0
148669,9a3900be-4326-4ca9-b155-bc85d943e892,2025-04-12,198.0
148670,39aa39d9-ac0d-4b8e-ac3e-5cff6f6c3c8f,2025-04-11,673.0
148671,bec05f9e-81dd-45cf-aedd-be84c58320f9,2025-04-07,57.0


In [23]:
# Verify the result is same as above
print(f"Date range: {df_transactions_filter2['DATE'].min()} to {df_transactions_filter2['DATE'].max()}. Resulting dataset size: {len(df_transactions_filter2)}")

Date range: 2025-04-07 to 2025-04-13. Resulting dataset size: 148673


In [24]:
assert df_transactions_filter2.get_backend() == "Pandas" 

In [25]:
len(df_transactions_filter2)

148673

Once you are in pandas, you can still continue to perform the same operations: 

In [26]:
%time
df_transactions_filter1.groupby("DATE").sum()["REVENUE"]

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 11 µs
Switcheroo operation: sum cost to move to pandas: 37 cost to stay: 750


Transferring data from Snowflake to Pandas ...:   0%|          | 0/2 [00:00<?, ?it/s]

DATE
2025-04-07    10653997.0
2025-04-08    10768178.0
2025-04-09    10685865.0
2025-04-10    10759502.0
2025-04-11    10663488.0
2025-04-12    10678733.0
2025-04-13    10841435.0
Freq: None, Name: REVENUE, dtype: float64

In [27]:
%time
df_transactions_filter2.groupby("DATE").sum()["REVENUE"]

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.11 µs


DATE
2025-04-07    10653997.0
2025-04-08    10768178.0
2025-04-09    10685865.0
2025-04-10    10759502.0
2025-04-11    10663488.0
2025-04-12    10678733.0
2025-04-13    10841435.0
Name: REVENUE, dtype: float64

### 💡 Automatic switching means that pandas work well for both small and large data

## Example 3: Performing Custom `apply` on small dataset

Forecast using last year's transaction data via a custom apply function

In [28]:
start_date = pd.Timestamp("2025-10-01")
end_date = pd.Timestamp("2025-10-31")

In [29]:
# Forecasting function using df.apply

def forecast_revenue(df, start_date, end_date):
    # Filter data from last year
    df_filtered = df[(df["DATE"] >= start_date - pd.Timedelta(days=365)) & (df["DATE"] < start_date)]
    
    # Append future dates to daily_avg for prediction
    future_dates = pd.date_range(start=start_date, end=end_date, freq="D")
    df_future = pd.DataFrame({"DATE": future_dates})

    # Group by DATE and calculate the mean revenue
    daily_avg = df_filtered.groupby("DATE")["REVENUE"].mean().reset_index()
    daily_avg["DATE"] = daily_avg["DATE"].astype('datetime64[ns]')
    # Merge future dates with predicted revenue, filling missing values
    df_forecast = df_future.merge(daily_avg, on="DATE", how="left")
    
    import numpy as np
    # Fill missing predicted revenue with overall mean from last year
    df_forecast["PREDICTED_REVENUE"] = np.nan
    df_forecast["PREDICTED_REVENUE"].fillna(daily_avg["REVENUE"].mean(), inplace=True)
    df_forecast["PREDICTED_REVENUE"] = df_forecast["PREDICTED_REVENUE"].astype("float")
    return df_forecast

In [30]:
df_forecast = forecast_revenue(df_transactions, start_date, end_date)
len(df_forecast)

Switcheroo operation: mean cost to move to pandas: 0 cost to stay: 750


Transferring data from Snowflake to Pandas ...:   0%|          | 0/2 [00:00<?, ?it/s]

31

The resulting dataframe is very small, since it is only the 1-month window we're performing forecast on.

In [31]:
assert df_forecast.get_backend() == 'Pandas'

In [32]:
def adjust_for_holiday_weekend(row):
    # For national holidays, revenue down 5% since stores are closed. For weekends, revenue is up 5% due to increased activity.
    if row["DATE"].strftime('%Y-%m-%d') in list(df_us_holidays["Date"].dt.strftime('%Y-%m-%d')): 
        return row["PREDICTED_REVENUE"] * 0.95
    elif row["DATE"].weekday() == 5 or row["DATE"].weekday() == 6: #Saturday/Sundays
        return row["PREDICTED_REVENUE"] * 1.05
    return row["PREDICTED_REVENUE"]

In [33]:
# Adjust for holidays using the apply function
df_forecast["PREDICTED_REVENUE"] = df_forecast.apply(adjust_for_holiday_weekend, axis=1)
df_forecast[["DATE","PREDICTED_REVENUE"]]

,DATE,PREDICTED_REVENUE
0,2025-10-01,505.059861
1,2025-10-02,505.059861
2,2025-10-03,505.059861
3,2025-10-04,530.312854
4,2025-10-05,530.312854
5,2025-10-06,505.059861
6,2025-10-07,505.059861
7,2025-10-08,505.059861
8,2025-10-09,505.059861
9,2025-10-10,505.059861


In [34]:
assert df_forecast.get_backend() == 'Pandas'

In [35]:
print(f"Altair takes in {type(df_forecast)} with {df_forecast.get_backend()} as backend, since we implement the dataframe interchange protocol")

Altair takes in <class 'modin.pandas.dataframe.DataFrame'> with Pandas as backend, since we implement the dataframe interchange protocol


In [36]:
import altair as alt
alt.data_transformers.disable_max_rows()

chart_predicted = alt.Chart(df_forecast).mark_line(color='blue').encode(
    x='monthdate(DATE):T',
    y=alt.Y('PREDICTED_REVENUE:Q',scale=alt.Scale(domain=[470, 550])),
    tooltip=['DATE', 'PREDICTED_REVENUE']
)
chart_predicted

ValueError: invalid literal for int() with base 10: ''

alt.Chart(...)

In [37]:
df_transactions_filtered = df_transactions[
    (df_transactions["DATE"] >= start_date - pd.Timedelta(days=365)) &
    (df_transactions["DATE"] < end_date - pd.Timedelta(days=365))
]
df_transactions_filtered_groupby = df_transactions_filtered.groupby("DATE")["REVENUE"].mean().reset_index()

In [38]:
print(f"Altair takes in {type(df_transactions_filtered_groupby)} with {df_transactions_filtered_groupby.get_backend()} as backend, since we implement the dataframe interchange protocol")

Altair takes in <class 'modin.pandas.dataframe.DataFrame'> with Snowflake as backend, since we implement the dataframe interchange protocol


In [39]:
df_forecast_labeled = df_forecast.copy()
df_forecast_labeled['Label'] = 'Predicted Revenue'
df_forecast_labeled = df_forecast_labeled.rename(columns={'PREDICTED_REVENUE': 'Value'})

df_last_year_labeled = df_transactions_filtered_groupby.copy()
df_last_year_labeled['Label'] = 'Revenue'
df_last_year_labeled = df_last_year_labeled.rename(columns={'REVENUE': 'Value'})

# Combine
combined_df = pd.concat([
    df_forecast_labeled[['DATE', 'Value', 'Label']],
    df_last_year_labeled[['DATE', 'Value', 'Label']]
])

# Plot with Value on X and color based on Label
final_chart = alt.Chart(combined_df).mark_line().encode(
    y=alt.Y('Value:Q',scale=alt.Scale(domain=[470, 550])),
    x='monthdate(DATE):T',
    color=alt.Color('Label:N', legend=alt.Legend(title='Type')),
    tooltip=['DATE', 'Value', 'Label']
).properties(
    title='Revenue vs Predicted Revenue (by Value)'
)

final_chart

alt.Chart(...)

### 💡 Apply on small dataset is much faster with automatic switching running with pandas locally.